# Problem statement
- OBJECTIVE: Forecast customer demand based on historical sales and related data from a retailer.
             The goal of this competition is to develop a model that predicts customer demand for a given set of products across multiple stores. Your predictions will help optimize stock management and reduce operational inefficiencies.

- KEY QUESTIONS: What is the goal?
                 What will the success criteria? The competition uses Root Mean Squared Error (RMSE) as the evaluation metric.


# Data Understanding
- Data description: The dataset consists of sales information, price changes, promotions, markdowns, and product catalog details.
- Data Sources: Kaggle - https://www.kaggle.com/competitions/ml-zoomcamp-2024-competition/data 
- Data timeline : 25 months of sales data from four stores and predict the demand for products in the next month. 

# Explore data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import argparse

# Function to display csv file

In [ ]:
sales_df = pd.read_csv('D:/Project Phase 5/Data/ml-zoomcamp-2024-competition/sales.csv')
sales_df.head(10)

In [ ]:
online_df = pd.read_csv('D:/Project Phase 5/Data/ml-zoomcamp-2024-competition/online.csv')
online_df.head(10)

In [ ]:
markdown_df = pd.read_csv('D:/Project Phase 5/Data/ml-zoomcamp-2024-competition/markdowns.csv')
markdown_df.head(10)

In [ ]:
price_history_df = pd.read_csv('D:/Project Phase 5/Data/ml-zoomcamp-2024-competition/price_history.csv')
price_history_df.head(10)

In [ ]:
discount_history_df = pd.read_csv('D:/Project Phase 5/Data/ml-zoomcamp-2024-competition/discounts_history.csv')
discount_history_df.head(10)

In [ ]:
actual_matrix_df = pd.read_csv('D:/Project Phase 5/Data/ml-zoomcamp-2024-competition/actual_matrix.csv')
actual_matrix_df.head(10)

In [ ]:
print(f"Sales done in person",sales_df.isna().sum())
print(f"Sales online",online_df.isna().sum())
print(f"markdown",markdown_df.isna().sum())
print(f"Price history",price_history_df.isna().sum())
print(f"Discount history",discount_history_df.isna().sum())
print(f"Actual Matrix",actual_matrix_df.isna().sum())

In [ ]:
print(f'Sales dtype',sales_df.info())
print(f'Online dtype',online_df.info())
print(f'Markdown dtype',markdown_df.info())
print(f'Price History dtype',price_history_df.info())
print(f'Discount History dtype',discount_history_df.info())
print(f'Actual Matrix dtype',actual_matrix_df.info())

# Potential biases or imbalances

In [ ]:
# Function to check potential biases or imbalances
def check_biases_imbalances(df, dataset_name):
    print(f"Dataset: {dataset_name}")
    print("Missing values:")
    print(df.isna().sum())
    print("\nColumn value counts (for categorical columns):")
    for col in df.select_dtypes(include=['object', 'category']).columns:
        print(f"\n{col} value counts:")
        print(df[col].value_counts().head(10))  # Display top 10 most frequent values
    print("\nSummary statistics (for numerical columns):")
    print(df.describe())
    print("\n" + "-"*50 + "\n")

# Check biases or imbalances for each dataset
check_biases_imbalances(sales_df, "Sales DataFrame")
check_biases_imbalances(online_df, "Online Sales DataFrame")
check_biases_imbalances(markdown_df, "Markdown DataFrame")
check_biases_imbalances(price_history_df, "Price History DataFrame")
check_biases_imbalances(discount_history_df, "Discount History DataFrame")
check_biases_imbalances(actual_matrix_df, "Actual Matrix DataFrame")

In [ ]:
# Plot distribution of store_id across datasets
plt.figure(figsize=(12, 6))
sns.countplot(x='store_id', data=sales_df)
plt.title('Distribution of Store IDs in Sales Data')
plt.xlabel('Store ID')
plt.ylabel('Count')
plt.show()

plt.figure(figsize=(12, 6))
sns.countplot(x='store_id', data=online_df)
plt.title('Distribution of Store IDs in Online Sales Data')
plt.xlabel('Store ID')
plt.ylabel('Count')
plt.show()

# Plot distribution of item_id in markdown_df
plt.figure(figsize=(12, 6))
top_items = markdown_df['item_id'].value_counts().head(10)
sns.barplot(x=top_items.index, y=top_items.values)
plt.title('Top 10 Most Frequent Items in Markdown Data')
plt.xlabel('Item ID')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

# Plot distribution of promo_type_code in discount_history_df
plt.figure(figsize=(12, 6))
sns.countplot(x='promo_type_code', data=discount_history_df)
plt.title('Distribution of Promo Type Codes in Discount History Data')
plt.xlabel('Promo Type Code')
plt.ylabel('Count')
plt.show()

# Plot distribution of quantity in sales_df
plt.figure(figsize=(12, 6))
sns.histplot(sales_df['quantity'], bins=50, kde=True)
plt.title('Distribution of Quantity in Sales Data')
plt.xlabel('Quantity')
plt.ylabel('Frequency')
plt.show()

# Plot distribution of price in price_history_df
plt.figure(figsize=(12, 6))
sns.histplot(price_history_df['price'], bins=50, kde=True)
plt.title('Distribution of Price in Price History Data')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.show()

# Checking how much the data influences each other

In [ ]:
# Function to plot correlation matrix
def plot_correlation_matrix(df, title):
    plt.figure(figsize=(10, 8))
    correlation_matrix = df.corr()
    sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
    plt.title(title)
    plt.show()

# Plot correlation matrix for each dataset
plot_correlation_matrix(sales_df.select_dtypes(include=['float64', 'int64']), "Correlation Matrix: Sales Data")
plot_correlation_matrix(online_df.select_dtypes(include=['float64', 'int64']), "Correlation Matrix: Online Sales Data")
plot_correlation_matrix(markdown_df.select_dtypes(include=['float64', 'int64']), "Correlation Matrix: Markdown Data")
plot_correlation_matrix(price_history_df.select_dtypes(include=['float64', 'int64']), "Correlation Matrix: Price History Data")
plot_correlation_matrix(discount_history_df.select_dtypes(include=['float64', 'int64']), "Correlation Matrix: Discount History Data")
plot_correlation_matrix(actual_matrix_df.select_dtypes(include=['float64', 'int64']), "Correlation Matrix: Actual Matrix Data")

In [ ]:
import sqlite3

# Create a SQLite database
conn = sqlite3.connect('retail_data.db')
cursor = conn.cursor()

# Create tables
cursor.execute('''
CREATE TABLE sales (
    id INTEGER PRIMARY KEY,
    date TEXT,
    item_id TEXT,
    quantity REAL,
    price_base REAL,
    sum_total REAL,
    store_id INTEGER
)
''')

cursor.execute('''
CREATE TABLE online_sales (
    id INTEGER PRIMARY KEY,
    date TEXT,
    item_id TEXT,
    quantity REAL,
    price_base REAL,
    sum_total REAL,
    store_id INTEGER
)
''')

cursor.execute('''
CREATE TABLE markdowns (
    id INTEGER PRIMARY KEY,
    date TEXT,
    item_id TEXT,
    normal_price REAL,
    price REAL,
    quantity REAL,
    store_id INTEGER
)
''')

cursor.execute('''
CREATE TABLE price_history (
    id INTEGER PRIMARY KEY,
    date TEXT,
    item_id TEXT,
    price REAL,
    code INTEGER,
    store_id INTEGER
)
''')

cursor.execute('''
CREATE TABLE discount_history (
    id INTEGER PRIMARY KEY,
    date TEXT,
    item_id TEXT,
    sale_price_before_promo REAL,
    sale_price_time_promo REAL,
    promo_type_code REAL,
    doc_id TEXT,
    number_disc_day REAL,
    store_id INTEGER
)
''')

cursor.execute('''
CREATE TABLE actual_matrix (
    id INTEGER PRIMARY KEY,
    item_id TEXT,
    date TEXT,
    store_id INTEGER
)
''')

# Insert data into tables
sales_df.to_sql('sales', conn, if_exists='replace', index=False)
online_df.to_sql('online_sales', conn, if_exists='replace', index=False)
markdown_df.to_sql('markdowns', conn, if_exists='replace', index=False)
price_history_df.to_sql('price_history', conn, if_exists='replace', index=False)
discount_history_df.to_sql('discount_history', conn, if_exists='replace', index=False)
actual_matrix_df.to_sql('actual_matrix', conn, if_exists='replace', index=False)

# Merge datasets with similarities
# Example: Merge sales and online_sales on item_id, date, and store_id
cursor.execute('''
CREATE VIEW merged_sales AS
SELECT 
    s.id AS sales_id,
    o.id AS online_sales_id,
    s.date,
    s.item_id,
    s.quantity AS sales_quantity,
    o.quantity AS online_quantity,
    s.price_base AS sales_price_base,
    o.price_base AS online_price_base,
    s.sum_total AS sales_sum_total,
    o.sum_total AS online_sum_total,
    s.store_id
FROM sales s
LEFT JOIN online_sales o
ON s.item_id = o.item_id AND s.date = o.date AND s.store_id = o.store_id
''')

# Example: Merge markdowns and price_history on item_id and store_id
cursor.execute('''
CREATE VIEW merged_markdowns_price_history AS
SELECT 
    m.id AS markdown_id,
    p.id AS price_history_id,
    m.date AS markdown_date,
    p.date AS price_history_date,
    m.item_id,
    m.normal_price,
    m.price AS markdown_price,
    p.price AS price_history_price,
    m.store_id
FROM markdowns m
LEFT JOIN price_history p
ON m.item_id = p.item_id AND m.store_id = p.store_id
''')

# Commit changes and close connection
conn.commit()
conn.close()

To determine the best target and features for implementing a predictive model, 
we need to consider the problem statement and the available data. 
The objective is to **forecast customer demand** for a given set of products across multiple stores. 
This involves predicting the quantity of items sold, which directly impacts stock management and operational efficiency.

### **Best Target**
The best target variable for this problem is:
- **`quantity`**: This variable represents the number of items sold for a given product (`item_id`) on a specific date (`date`) and store (`store_id`). 
Predicting `quantity` aligns with the goal of forecasting customer demand.

### **Best Features**
The best features should be selected based on their relevance to predicting `quantity`. 
Below is a detailed explanation of the potential features from the datasets:

#### **1. Temporal Features**
- **`date`**: The date of the transaction is critical for identifying trends, seasonality, and patterns in sales. For example, sales may increase during holidays or weekends.
    - **Derived Features**: Extract day, month, year, day of the week, and whether the date is a holiday or weekend.

#### **2. Product Features**
- **`item_id`**: The unique identifier for each product. Different products may have different demand patterns.
    - **Derived Features**: Group products into categories (if available) or use historical sales data to calculate average sales per product.

#### **3. Store Features**
- **`store_id`**: The unique identifier for each store. Different stores may have varying customer bases and demand patterns.
    - **Derived Features**: Include store-specific attributes like location, size, or customer demographics (if available).

#### **4. Pricing Features**
- **`price_base`** (from `sales_df` and `online_df`): The base price of the product. Pricing directly influences demand.
- **`normal_price`** and **`price`** (from `markdown_df`): The normal price and discounted price of the product. Discounts can significantly impact sales.
- **`sale_price_before_promo`** and **`sale_price_time_promo`** (from `discount_history_df`): These provide insights into promotional pricing and its effect on demand.

#### **5. Promotion Features**
- **`promo_type_code`** (from `discount_history_df`): The type of promotion applied. Different promotions may have varying impacts on sales.
- **`number_disc_day`**: The number of days a discount was active. Longer promotions may lead to higher sales.

#### **6. Historical Sales Features**
- **`quantity`** (from `sales_df` and `online_df`): Historical sales data is crucial for forecasting future demand.
    - **Derived Features**: Calculate moving averages, rolling sums, and lag features (e.g., sales in the past 7 days, 30 days, etc.).

#### **7. Inventory Features**
- **`actual_matrix_df`**: This dataset provides information about the availability of products (`item_id`) in stores (`store_id`) on specific dates (`date`). Stock availability is a key factor in determining sales.

#### **8. External Factors**
- **Holidays and Events**: Incorporate external data about holidays, festivals, or special events that may influence demand.
- **Weather Data**: If available, weather conditions can also impact sales (e.g., increased demand for certain products during rainy or hot weather).

### **Feature Engineering**
To improve the predictive power of the model, additional features can be engineered:
1. **Lag Features**: Include lagged sales data (e.g., sales 1 week ago, 1 month ago).
2. **Cumulative Features**: Cumulative sales for a product or store over a specific period.
3. **Interaction Features**: Interaction between price and promotion (e.g., price * promo_type_code).
4. **Seasonality Features**: Identify seasonal trends (e.g., higher sales in summer or winter).

### **Feature Selection**
After feature engineering, statistical methods (e.g., correlation analysis, mutual information) or machine learning techniques (e.g., feature importance from tree-based models) can be used to select the most relevant features.

### **Conclusion**
- **Target**: `quantity`
- **Best Features**:
    - Temporal: `date` (with derived features like day, month, year, etc.)
    - Product: `item_id`
    - Store: `store_id`
    - Pricing: `price_base`, `normal_price`, `price`, `sale_price_before_promo`, `sale_price_time_promo`
    - Promotion: `promo_type_code`, `number_disc_day`
    - Historical Sales: Lagged and cumulative sales data
    - Inventory: Stock availability from `actual_matrix_df`
    - External: Holidays, events, and weather data (if available)

By combining these features, the model can effectively capture the factors influencing customer demand and provide accurate forecasts.